In [23]:
%env dataset_home=/Users/zhaoxuefeng/Documents/Project/zxfMLtools_Tutorial

env: dataset_home=/Users/zhaoxuefeng/Documents/Project/zxfMLtools_Tutorial


In [1]:
from zxfMLtools.datasets import Dataset

In [7]:
# dataset

In [6]:
# %load /Users/zhaoxuefeng/GitHub/zxfMLtools/zxfMLtools/datasets.py
"""
datasets
import matplotlib.pyplot as plt


dataset 主司 数据集的管理 方便的存储 管理 使用现有的主流数据集 或者自定义数据集
应该可以快速的把 文件 零散的表 组装成一个数据集
dataset info  制定好合适的下游  适合什么样的模型 什么样的模型不适合
dataset evals 什么样的数据是好数据 什么样的数据不是好数据
dataset clean 简单来说 就是将坏的数据变成好的数据
dataset manager ....



# 可完整运行



!pip install datasets transformers
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = processor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])


## dataset
dataset = tv.datasets.MNIST(".", download=True, transform=tv.transforms.ToTensor())
train, val = data.random_split(dataset, [55000, 5000])


"""
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import torch.utils.data as torch_data
import torchvision
import torch
from datasets import load_dataset_builder
from datasets import Dataset
from datasets import IterableDataset
from datasets import load_dataset
from datasets import get_dataset_config_names
from datasets import get_dataset_split_names
from datasets import DatasetDict
import os
from typing import List, Callable
from PIL import Image
import math

class datasethelp():
    def __init__(self):
        """
        maps
        iterable
        set_transform
        env
        totorch
        save
        other

        """
        self.maps = """
from torchvision import transforms
transform = transforms.Resize((224, 224))

def maps(example):
    example['img'] = transform(example['img'])
    return example

das = dataset.map(maps)# batched=True
"""
        self.iterable = """
datas.dataset['train'].to_iterable_dataset()
"""
        self.set_transform = """
dataset.set_transform(transforms)     
"""
        self.env = """dataset_home"""
        self.totorch = """ds.with_format("torch", device=device)"""
        self.save = """
.save_to_disk("my_dataset")
.push_to_hub("<username>/my_dataset")
        """
        self.other = """
import os
from datasets.distributed import split_dataset_by_node
ds = split_dataset_by_node(ds, rank=int(os.environ["RANK"]), world_size=int(os.environ["WORLD_SIZE"]))  
        """

    def __repr__(self):
        return """
        maps
        iterable
        set_transform
        env
        totorch
        save
        other

        """


DatasetDict.help = datasethelp()


## HELP ##


class TDataset(torch_data.Dataset):
    def __init__(self, data: pd.DataFrame, target: List = None, pretransform=None, transform=None, window=1,
                 type='stepby', info=None):
        """
        data: pd.DataFrame,target:List=None,pretransform=None,transform=None, window=1, type='stepby',info = None):

        :param data:
        :param target:
        :param pretransform:
        :param transform:
        :param window:
        :param type:
        :param info:
        """
        super().__init__()
        if pretransform is not None:
            data = pretransform.fit_transform(data)
        self.data = data
        self.window = window
        self.type = type
        if window == 1:
            self.data_list = self.data
        else:
            self.data_list = self.data  # self.data_list = devide_data(self.data, window, type)
        self.transform = transform
        if target is not None:
            assert len(target) == len(self.data_list)
            self.target = pd.DataFrame(target)
        else:
            cond = [0] * len(self.data_list)
            self.target = pd.DataFrame(cond)

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.window == 1:
            datas = self.data_list.iloc[idx]
        else:
            datas = self.data_list[idx]
        conds = self.target.iloc[idx]
        if self.transform is not None:
            datas = self.transform(datas)
        return datas, conds.values


class Dataset():
    def __new__(cls, *args, mode='use', **kwargs):
        dataset_home = os.environ.get('dataset_home', '/qe/data/data/Dataset_LLM')
        try:
            path = args[0]
            if not path.startswith('/'):
                path = os.path.join(dataset_home, path)
            args = (path,) + args[1:]
        except IndexError as e:
            path = kwargs['path']
            if not path.startswith('/'):
                path = os.path.join(dataset_home, path)
            kwargs['path'] = path

        if mode == 'info':
            instance = super().__new__(cls)
            return instance
        elif mode == 'use':
            dataset = load_dataset(*args, **kwargs)
            return dataset

    def __init__(self, path: str, **kwargs):
        """
        "totto"
        dataset_home
        """
        self.configs = get_dataset_config_names(path)
        self.split_names = get_dataset_split_names(path)

    def __repr__(self):
        return f"configs: {self.configs}" + '\n' + f"split_name: {self.split_names}" + '\n' + "torch.Size([1, 32, 32, 3]): batch: w:h:channel"

    @staticmethod
    def from_iter(iters):

        dataset = IterableDataset.from_generator(iters)
        return dataset

    @staticmethod
    def from_dict(dicts):
        dataset = Dataset.from_dict(dicts)
        return dataset

    @staticmethod
    def from_builder(builder):
        dataset = load_dataset_builder(builder)
        return dataset


class DataLoaders():
    def __init__(self, dataset, batch_size=32, shuffle=True):
        self.dataset = dataset
        self.loader = DataLoader(self.dataset, batch_size=batch_size, shuffle=shuffle)




class Vision():
    def __init__(self):
        pass

    @staticmethod
    def _find_dimensions(n):
        # 开始时，我们假设最佳的宽度和高度都是n
        width, height = n, 1
        # 我们从1开始，一直到sqrt(n)，寻找能够被n整除的数
        for i in range(1, math.isqrt(n) + 1):
            if n % i == 0:
                # 如果i能被n整除，那么我们就找到了一个更好的宽度和高度
                width, height = n // i, i
        return width, height

    @staticmethod
    def _image_grid(imgs, rows, cols):
        # assert len(imgs) == rows*cols
        w, h = imgs[0].size
        grid = Image.new('RGB', size=(cols * w, rows * h))
        grid_w, grid_h = grid.size
        for i, img in enumerate(imgs):
            grid.paste(img, box=(i % cols * w, i // cols * h))
        return grid

    @staticmethod
    def _tensor_grid(tensors):
        # 假设你有一个形状为[9, 3, 32, 32]的张量
        # 使用torchvision.utils.make_grid创建一个图像网格
        grid = torchvision.utils.make_grid(tensors, nrow=3)
        # 转换颜色空间并显示图像
        return np.transpose(grid, (1, 2, 0))

    @staticmethod
    def _show(grid):
        plt.imshow(grid)
        plt.show()

    @staticmethod
    def run(datas: 'Tensor' or 'data'):
        if isinstance(datas, torch.Tensor):
            grid = super()._tensor_grid(datas)
        else:
            imgs = datas['img']
            labels = datas['label']
            assert len(imgs) == len(labels)
            n = len(imgs)
            width, height = super()._find_dimensions(n)
            grid = super()._image_grid(imgs, rows=height, cols=width)
            label_grid = np.array(labels).reshape(height, width)
            print(label_grid)
        super().show(grid)


if __name__ == '__main__':
    datas = Dataset("/qe/data/data/Dataset_LLM/imdb")

In [ ]:
Dataset('cifar10')

FileNotFoundError: Couldn't find a dataset script at /Users/zhaoxuefeng/Documents/Project/zxfMLtools_Tutorial/cifar10/cifar10.py or any data file in the same directory.

In [ ]:
dataset = Dataset('cifar10')

('/Datasets/cifar10',) {}


HF google storage unreachable. Downloading and preparing it from source

KeyboardInterrupt

